# Mein Projekt

Erkenntnisse
-----
- Abstand zwischen zwei Punkten im Raster ~48 (da dst(ox, origin) & dst(oy, origin) ~144)
- Abstand zwischen zwei Punkten im Raster, diagonal = sqrt(48^2 + 48^2) ~ 68, mit Messwerten verifizierbar
- Abweichung von theoretischer zu echter Distanz o,x bzw. o,y ~10%

ToDo
----
- test for different touchcodes

In [103]:
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull
from scipy.spatial import distance

samples = {}
samples[0b10000000] = [[(1643,614),(1450,649),(1555,663),(1568,731)],
           [(577,629),(379,577),(471,631),(453,701)],
           [(1486,68),(1638,213), (1581,119),(1628,69)],
           [(1676,651),(1530,799),(1619,750),(1675,791)],
           [(176,469), (320,324),(225,373),(175,324)],            
           [(725,544),(860,697),(819,599),(869,557)],
           [(346,509),(494,368),(399,415),(346,367)],
           [(769,593),(750,792),(793,695),(865,701)],
            [(269,202),(477,205),(375,164),(382,93)],
            [(303,152), (379,577),(368,171),(368,285)]]

def sample_to_ndarray(sample):
    points = []
    
    for point in sample:
        points.append(point[0])
        points.append(point[1])
    
    return np.array(points)


def render_sample(index):
    df = pd.DataFrame(samples[index], columns=['x', 'y'])
    print(df)
    fig = df.plot.scatter(x='x', y='y', color='red', figsize=(15,15))

    fig.set_xlim([0, 2000])
    fig.set_ylim([0, 2000])

    plt.gca().set_aspect('equal', adjustable='box')
    plt.ylabel('some numbers')

    plt.show()

def convex_hull(sample):
    # convert points of sample to ndarray
    points = np.asarray(sample)
    
    # find the convex hull
    hull = ConvexHull(points)    
   
    plt.figure(num=None, figsize=(18, 16), dpi=320, facecolor='w', edgecolor='r')
    
    plt.xlim([0,2000]) 
    plt.ylim([0,2000]) 

    # plot the original points
    plt.plot(points[:, 0], points[:, 1], 'o')

    # plot the convex hull around it
    for simplex in hull.simplices:
        plt.plot(points[simplex, 0], points[simplex, 1], 'r')

    # adjustment to coordinate system
    plt.gca().set_aspect('equal', adjustable='box')
    
    # display that crap
    plt.show()


def get_distances_for_sample(sample):
    oy = sample[0]
    ox = sample[1]
    tc = sample[2] # part of the code
    origin = sample[3] # origin

    return[distance.euclidean(origin, ox), distance.euclidean(origin, oy),distance.euclidean(ox, oy),distance.euclidean(tc, origin) ]
    

def get_statistics_for_sampleset(sampleset):
    dst_origin_ox = []
    dst_origin_oy = []
    dst_ox_oy = []
    dst_origin_tc = []

    for i in range(0,9): 
        sample = samples[0b10000000][i]
        distances = get_distances_for_sample(sample)
        
        dst_origin_ox.append(distances[0])
        dst_origin_oy.append(distances[1])
        dst_ox_oy.append(distances[2])
        dst_origin_tc.append(distances[3])

    print("dst(origin,x): mean({0}), max({1}, min({2}))".format(np.mean(dst_origin_ox), np.max(dst_origin_ox), np.min(dst_origin_ox)))
    print("dst(origin,y): mean({0}), max({1}, min({2}))".format(np.mean(dst_origin_oy), np.max(dst_origin_oy), np.min(dst_origin_oy)))
    print("dst(ox,oy): mean({0}), max({1}, min({2}))".format(np.mean(dst_ox_oy), np.max(dst_ox_oy), np.min(dst_ox_oy)))
    print("dst(origin,tc): mean({0}), max({1}, min({2}))".format(np.mean(dst_origin_tc), np.max(dst_origin_tc), np.min(dst_origin_tc)))

def get_orientation_marks(sample):
    points = np.asarray(sample)   
    hull = ConvexHull(points)  
    vertices = hull.vertices

    #continue: distances for all combinations, find the right ones.
    combinations = list(itertools.combinations(vertices, 2))
    
    for combination in combinations:
        p1 = points[combination[0]]
        p2 = points[combination[1]]
        
        dst = distance.euclidean(p1, p2)
        
        print("p1: {0}, p2: {1}, dst: {2}".format(p1, p2, dst))
        
        # wolf tipp: distanz (x,y) = längste distanz = lmax
        # distanz (o,x) || (o,y) = sqrt(lmax)
        
    
sample = samples[0x80][4]
get_statistics_for_sampleset(samples[0b10000000])

#get_orientation_marks(sample)
#convex_hull(sample)
#get_statistics_for_sampleset(samples[0x80])

def norm(sample):
    o = np.array(sample[3])
    x = np.array(sample[1])
    y = np.array(sample[0])
    s = np.array(sample[2])
    
    # Richtungsvektoren entlang der Kanten vom Referenzsystem
    vx = x - o
    vy = y - o
    # Ortsvektor des Punkts bzgl o (wo ist s bgzl des neuen Ursprungs o)
    so = s-o
    
    # Normierung der Richtungsvektoren    
    vx = (vx/(np.linalg.norm(vx)))/(np.linalg.norm(vx))*3
    vy = (vy/(np.linalg.norm(vy)))/(np.linalg.norm(vy))*3
   
    xcor = np.dot(vx,so)
    ycor = np.dot(vy,so)
    
    
    dst_x_y = distance.euclidean(x,y)
    dst_o_xy = dst_x_y / np.sqrt(2)
    dst_o_x = distance.euclidean(o,x)
    dst_o_y = distance.euclidean(o,y)
    
    
    print("s.x: {0}, s.y: {1}".format(xcor, ycor))
    print("next step. dst(x,y) = {0} => dst(o,x|y) ~= {1} [is {2} and {3}]".format(dst_x_y, dst_o_xy, dst_o_x, dst_o_y))
    
for i in range(0, 9):
    sample = samples[0x80][i]
    norm(sample)


dst(origin,x): mean(144.94101688140265), max(148.00337833982033, min(140.28898745090436))
dst(origin,y): mean(144.1623199552579), max(157.00318468107582, min(138.97481786280562))
dst(ox,oy): mean(204.39613129984087), max(210.06903627141244, min(196.1479033790573))
dst(origin,tc): mean(69.99344507785503), max(72.27724399837061, min(65.29931086925804))
s.x: 1.033029833397908, s.y: 1.0843429636533086
next step. dst(x,y) = 196.1479033790573 => dst(o,x|y) ~= 138.69751259485514 [is 143.69411957348845 and 138.97481786280562]
s.x: 1.0571647803568003, s.y: 1.0610894941634241
next step. dst(x,y) = 204.71443525066815 => dst(o,x|y) ~= 144.75496537252184 [is 144.40221604947757 and 143.38758663147937]
s.x: 0.9689959685160301, s.y: 0.985469873543268
next step. dst(x,y) = 210.06903627141244 => dst(o,x|y) ~= 148.54124006483855 [is 144.34680460612904 and 142.00352108310554]
s.x: 1.1084451609843993, s.y: 0.8699556145094638
next step. dst(x,y) = 207.8942038634074 => dst(o,x|y) ~= 147.0034013211939 [is 145